# Generate data
- Target: generate n (default n = 24) rows for each stay_id, with the following 30 columns
    - baseline (9 cols): age, gender, insurance type, race, first care unit ICU, Admission type, Height, Weight, Tobacco
    - ventilation (9 cols): peep, fio2, tidal_volume_observed, ~~O2 flow~~ (drop due to too many missing), respiratory_rate_set, Plateau pressure, RSBI: resp_rate / tidal_volume_observed, minute_ventilation: resp_rate * tidal_volume_observed, ventilator_mode, ventilator_mode_group
    - vitalsign (6 cols): heart_rate, sbp, dbp, mbp, spO2, resp_rate
    - id_info (6 cols): subject_id, stay_id, hadm_id, charttime, before_weaning_hr, label
- Baseline features steps:
    1. Read in data
    2. Merge with ground truth ("label")
    3. Show tableone for the Baseline features with missing value and outliers
    4. Use KNN for the missing value on weight and height
    5. Deal with outlier by 3 std as boundary
    6. Show tableone for the Baseline features after deal with outliers
- Charttime featrues steps:
    1. Round the charttime to hour
    2. Filter the charttime data start / end from ventilation starttime / endtime within 24 hr
    3. Fill the missing value forward then backward by stay_id by each of the charttime df (ventilator_setting_df, vitalsign_df, labevents_df)
    4. Generate template: n rows for each stay_id, # of total rows = n * # of stay_id
    5. Merge template with charttime dataframes (left join): some hour might be empty, while some hour might have more than one rows
    6. Fill the missing value again (some missing value might appear when step 5. merge into 24 rows)
    7. Use aggregation function dealing with "one hour with multiple line" (e.g., vitalsign: one hour have two records -> merge into one row)
    8. Calculate 'RSBI' and 'minute_ventilation' by 'tidal_volume_observed' and 'resp_rate'
    9. Merge with ground truth ("label")
    10. Show tableone for the Charttime features with missing value and outliers
    11. Deal with outlier by 3 std as boundary
    12. Show tableone for the Baseline features after deal with outliers

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.impute import KNNImputer
from tableone import TableOne, load_dataset
pd.options.mode.chained_assignment = None

## Baseline features

### Read in data

In [ ]:
baseline_df = pd.read_csv("data/data_by_table/baseline.csv")
ventilator_setting_df = pd.read_csv("data/data_by_table/mimiciv_derived_ventilator_setting.csv")
vitalsign_df = pd.read_csv("data/data_by_table/mimiciv_derived_vitalsign.csv")
# labevents_df = pd.read_csv("data/data_by_table/mimiciv_hosp_labevents.csv")
labevents_df = pd.read_csv("data/data_by_table/mimiciv_hosp_labevents_itemid_50821.csv")
cohort_subject_id_stay_id_df = pd.read_csv("data/data_by_table/cohort_subject_id_stay_id.csv")
ground_truth_df = pd.read_csv("data/data_by_table/ground_truth.csv")
labevents_df['O2_flow'].replace('___', pd.NA, inplace=True)

In [ ]:
baseline_df

In [ ]:
baseline_df.columns

In [ ]:
ground_truth_df

### Merge with ground truth ("label")

In [ ]:
baseline_gt_df = pd.merge(baseline_df, ground_truth_df[["stay_id", "label"]], on = ["stay_id"], how = "inner")
baseline_gt_df

### Show tableone for the Baseline features with missing value and outliers


In [ ]:
features_col = ['age_now', 'gender', 'insurance', 'admission_type',
                'first_careunit', 'weight_kg', 'height_cm', 'tobacco', 'label'] # 'race'
categorical = ['gender', 'insurance', 'admission_type', 'first_careunit']   # 'race'
groupby = ['label']
table_with_outlier = TableOne(baseline_gt_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_with_outlier

In [ ]:
features_col = ['race', 'label']
categorical = ['race']   # 'race'
groupby = ['label']
table_with_outlier = TableOne(baseline_gt_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_with_outlier

### Use KNN for the missing value on weight and height

In [ ]:
def KNN_weight_height(baseline_gt_df):
    imputer = KNNImputer(n_neighbors=3)
    baseline_gt_df[['height_cm', 'weight_kg']] = imputer.fit_transform(baseline_gt_df[['height_cm', 'weight_kg']])
    baseline_gt_df['height_cm'].fillna(baseline_gt_df.groupby('gender')['height_cm'].transform('mean'), inplace=True)
    baseline_gt_df['weight_kg'].fillna(baseline_gt_df.groupby('gender')['weight_kg'].transform('mean'), inplace=True)
    return baseline_gt_df
    # baseline_df_p.drop(columns=['subject_id', 'hadm_id'])

In [ ]:
baseline_fill_w_h_df = KNN_weight_height(baseline_gt_df)

In [ ]:
features_col = ['weight_kg', 'height_cm', 'label']
categorical = []
groupby = ['label']
table_with_outlier = TableOne(baseline_fill_w_h_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_with_outlier

### Deal with outlier by 3 std as boundary

In [ ]:
def handle_outliers(series, threshold=3):
    lower_bound = series.quantile(0.25) - threshold * (series.quantile(0.75) - series.quantile(0.25))
    upper_bound = series.quantile(0.75) + threshold * (series.quantile(0.75) - series.quantile(0.25))
    # print(f'upper_bound: {upper_bound}\n lower_bound: {lower_bound}')
    # print(series[series < lower_bound])
    # print(series[series > upper_bound])
    series[series < lower_bound] = lower_bound
    series[series > upper_bound] = upper_bound
    return series

In [ ]:
numeric_columns = baseline_fill_w_h_df.select_dtypes(include=['float64', 'int64']).columns
numeric_columns = numeric_columns.drop(["stay_id", "subject_id", 'hadm_id', 'age_now', 'tobacco', 'label'])
# Apply the function to each numeric column
baseline_fill_w_h_df[numeric_columns] = baseline_fill_w_h_df[numeric_columns].apply(handle_outliers)

### Show tableone for the Baseline features after deal with outliers

In [ ]:
features_col = ['weight_kg', 'height_cm', 'label']
categorical = []
groupby = ['label']
table_without_outlier = TableOne(baseline_fill_w_h_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_without_outlier

## Chattime features

### Round the charttime to hour

In [ ]:
def round_hour(charttime_df):
    charttime_df['charttime'] = pd.to_datetime(charttime_df['charttime']).dt.round('H')
    return charttime_df

In [ ]:
ventilator_setting_round_df = round_hour(ventilator_setting_df)
vitalsign_round_df = round_hour(vitalsign_df)
labevents_round_df = round_hour(labevents_df)

### Filter the charttime data start / end from ventilation starttime / endtime within 24 hr

In [ ]:
def filter_starttime_endtime(ground_truth_df, charttime_df, filter_hours):
    filter_starttime_endtime_df = pd.DataFrame()
    for index, row in ground_truth_df.iterrows():
        start_time = pd.to_datetime(row['starttime']) - timedelta(hours=filter_hours)
        end_time = pd.to_datetime(row['endtime']) + timedelta(hours=filter_hours)
        stay_id = row['stay_id']
        df_subset = charttime_df[(charttime_df['stay_id'] == stay_id) & (charttime_df['charttime'] >= start_time) & (charttime_df['charttime'] <= end_time)]
        filter_starttime_endtime_df = pd.concat([filter_starttime_endtime_df, df_subset])
    return filter_starttime_endtime_df

In [ ]:
print(len(ventilator_setting_df))
print(len(vitalsign_df))
print(len(labevents_df))

In [ ]:
filter_hours = 24
ventilator_setting_filter_df = filter_starttime_endtime(ground_truth_df, ventilator_setting_round_df, filter_hours)
vitalsign_filter_df = filter_starttime_endtime(ground_truth_df, vitalsign_round_df, filter_hours)
labevents_filter_df = filter_starttime_endtime(ground_truth_df, labevents_round_df, filter_hours)

In [ ]:
print(len(ventilator_setting_filter_df))
print(len(vitalsign_filter_df))
print(len(labevents_filter_df))

In [ ]:
filter_hours = 0
ventilator_setting_filter_0_df = filter_starttime_endtime(ground_truth_df, ventilator_setting_filter_df, filter_hours)
vitalsign_filter_0_df = filter_starttime_endtime(ground_truth_df, vitalsign_filter_df, filter_hours)
labevents_filter_0_df = filter_starttime_endtime(ground_truth_df, labevents_filter_df, filter_hours)

In [ ]:
print(len(ventilator_setting_filter_0_df))
print(len(vitalsign_filter_0_df))
print(len(labevents_filter_0_df))

### Fill the missing value forward then backward by stay_id by each of the charttime df (ventilator_setting_df, vitalsign_df, labevents_df)

In [ ]:
def fill_Nan(df):
    # Convert 'charttime' to datetime format for sorting
    df['charttime'] = pd.to_datetime(df['charttime'])
    # Sort DataFrame by 'stay_id' and 'charttime'
    df.sort_values(by=['stay_id', 'charttime'], inplace=True)
    # Fill NaN values within each 'stay_id' using forward-fill and backward-fill
    df = df.groupby('stay_id').apply(lambda group: group.ffill().bfill())
    
    return df

In [ ]:
ventilator_setting_fill_df = fill_Nan(ventilator_setting_filter_df)
vitalsign_fill_df = fill_Nan(vitalsign_filter_df)
labevents_fill_df = fill_Nan(labevents_filter_df)

### Generate template: n rows for each stay_id, # of total rows = n * # of stay_id

In [ ]:
def generate_df_template(stay_id, endtime):
    end_time = pd.to_datetime(endtime).floor('H')
    time_intervals = [end_time - timedelta(hours=i) for i in range(24)]
    df = pd.DataFrame(time_intervals, columns=['charttime'])
    df['stay_id'] = stay_id 
    return df

In [ ]:
def generate_all_template(ground_truth_df):
    data_template = pd.DataFrame()
    for index, row in ground_truth_df.iterrows():
        data_now = generate_df_template(row['stay_id'],row['endtime'])
        data_template = pd.concat([data_template, data_now], ignore_index=False)
    return data_template

In [ ]:
data_template = generate_all_template(ground_truth_df)

### Merge template with charttime dataframes (left join): some hour might be empty, while some hour might have more than one rows

In [ ]:
def merge_dfs(data_template, charttime_dfs):
    for df in charttime_dfs:
        df = df.drop(columns=["subject_id"])
        data_template = pd.merge(data_template, df, on=["stay_id", "charttime"], how="left")
    return data_template

In [ ]:
merge_charttime_df = merge_dfs(data_template, [ventilator_setting_fill_df, vitalsign_fill_df, labevents_fill_df])

### Fill the missing value again (some missing value might appear when step 5. merge into 24 rows)

In [ ]:
merge_charttime_fill_df = fill_Nan(merge_charttime_df)

### Filter endtime previous 24 hours

In [ ]:
def filter_pre_24_hr(ground_truth_df, charttime_df):
    pre_24_hr_df = pd.DataFrame()
    for index, row in ground_truth_df.iterrows():
        start_time = pd.to_datetime(row['endtime']) - timedelta(hours=24)
        end_time = pd.to_datetime(row['endtime'])
        stay_id = row['stay_id']
        df_subset = charttime_df[(charttime_df['stay_id'] == stay_id) & (charttime_df['charttime'] > start_time) & (charttime_df['charttime'] <= end_time)] # TODO: not sure < or <= end_time
        pre_24_hr_df = pd.concat([pre_24_hr_df, df_subset])
    return pre_24_hr_df

In [ ]:
filter_pre_24_hr_df = filter_pre_24_hr(ground_truth_df, merge_charttime_fill_df)

### Use aggregation function dealing with "one hour with multiple line" (e.g., vitalsign: one hour have two records -> merge into one row)

In [ ]:
def before_weaning_hr(df):
    # Assuming your DataFrame is named df
    df['charttime'] = pd.to_datetime(df['charttime'])  # Convert to datetime if not already
    df.sort_values(['stay_id', 'charttime'], inplace=True)  # Sort by stay_id and charttime

    # Calculate the 'before_weaning_hr' column
    df['before_weaning_hr'] = df.groupby('stay_id').cumcount(ascending=False)
def min_agg(series):
    non_nan_values = series.dropna()
    if len(non_nan_values) == 0:
        return np.nan  # No values available, keep it as NaN
    else:
        return non_nan_values.min()  # Choose the minimum value among available values
def max_agg(series):
    non_nan_values = series.dropna()
    if len(non_nan_values) == 0:
        return np.nan  # No values available, keep it as NaN
    else:
        return non_nan_values.max()  # Choose the maximum value among available values
def HR_agg(series): # this rule is by Dr.
    # Rule 1: If all records are greater than or equal to 80, return the max
    if all(value >= 80 for value in series):
        return series.max()
    # Rule 2: If all records are less than 80, return the min
    elif all(value < 80 for value in series):
        return series.min()
    # Rule 3: Otherwise, return the min
    else:
        return series.min()
def RR_agg(series): # this rule is by Dr.
    # Rule 1: If all records are greater than or equal to 12, return the max
    if all(value >= 12 for value in series):
        return series.max()
    # Rule 2: If all records are less than 12, return the min
    elif all(value < 12 for value in series):
        return series.min()
    # Rule 3: Otherwise, return the min
    else:
        return series.min()

def flat_to_24_rows(pre_24_hr_df):
    # Apply the custom aggregation function for each feature
    pre_24_rows_df = pre_24_hr_df.groupby(['stay_id', 'charttime']).agg({
        'peep': max_agg,
        'fio2': max_agg,
        'tidal_volume_observed': max_agg,
        'respiratory_rate_set': max_agg,
        'plateau_pressure': max_agg,
        'heart_rate': HR_agg,
        'sbp': min_agg,
        'dbp': min_agg,
        'mbp': min_agg,
        'resp_rate': RR_agg,
        'spo2': min_agg,
        'O2_flow': max_agg,
        'ventilator_mode': 'first'
    }).reset_index()
    before_weaning_hr(pre_24_rows_df)
    return pre_24_rows_df

In [ ]:
pre_24_rows_df = flat_to_24_rows(filter_pre_24_hr_df)

### Calculate 'RSBI' and 'minute_ventilation' by 'tidal_volume_observed' and 'resp_rate'

In [ ]:
def categorize_ventilator_mode(ventilator_mode):
    complete_support = ["PRVC/AC", "PCV+Assist", "PCV+", "MMV/AutoFlow", "APRV", "CMV/AutoFlow",
                        "CMV", "PRES/AC (PCAC)", "APV (cmv)", "PRVC/SIMV (=aprv)", "MMV",
                        "VOL/AC", "APRV/Biphasic+ApnVol", "APRV/Biphasic+ApnPress", "(S) CMV",
                        "P-CMV", "CMV/ASSIST", "MMV/PSV/AutoFlow", "CMV/ASSIST/AutoFlow"]

    partial_support = ["SIMV/PSV/AutoFlow", "SIMV/PRES", "SIMV/PSV", "SIMV/AutoFlow", "SIMV/VOL",
                       "SIMV", "SYNCHRON MASTER", "SYNCHRON SLAVE"]

    minimal_support = ["CPAP/PSV+ApnVol", "CPAP/PPS", "PCV+/PSV", "Apnea Ventilation", "CPAP",
                       "MMV/PSV", "SPONT", "CPAP/PSV+ApnPres", "Ambient", "CPAP/PSV+Apn TCPL(time cycle pressure limit)",
                       "null", "PSV/SBT", "Standby", "CPAP/PSV"]

    if ventilator_mode in complete_support:
        return "Complete Support"
    elif ventilator_mode in partial_support:
        return "Partial Support"
    elif ventilator_mode in minimal_support:
        return "Minimal Support"
    else:
        return "Minimal Support"

In [ ]:
pre_24_rows_df['RSBI'] = pre_24_rows_df['resp_rate'] / (pre_24_rows_df['tidal_volume_observed'] * 0.001)
pre_24_rows_df['minute_ventilation'] = pre_24_rows_df['resp_rate'] * (pre_24_rows_df['tidal_volume_observed'] * 0.001)
# Apply the categorization function to create the new column
pre_24_rows_df['ventilator_mode_group'] = pre_24_rows_df['ventilator_mode'].apply(categorize_ventilator_mode)

In [ ]:
# pre_24_rows_df[pre_24_rows_df["ventilator_mode_group"] == "Minimal Support"]
pre_24_rows_df[pre_24_rows_df["RSBI"] > 105]

### Merge with ground truth ("label")

In [ ]:
pre_24_rows_gt_df = pd.merge(pre_24_rows_df, ground_truth_df[["stay_id", "label"]], on=["stay_id"], how="inner")
pre_24_rows_gt_df.reset_index(drop=True, inplace=True)

In [ ]:
pre_24_rows_gt_df

### Show tableone for the Charttime features with missing value and outliers

In [ ]:
features_col = ['peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 
                'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 
                'resp_rate', 'spo2', 'O2_flow', 'RSBI', 'minute_ventilation', 'label']
categorical = []
groupby = ['label']
table_with_outlier = TableOne(pre_24_rows_gt_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_with_outlier

### Visualization for the outliers obervation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming pre_24_rows_gt_df is your DataFrame
# Drop non-numeric columns for simplicity in the example
numeric_cols = pre_24_rows_gt_df.select_dtypes(include='number').columns

# Set up subplots
fig, axes = plt.subplots(nrows=len(numeric_cols), ncols=1, figsize=(10, 5 * len(numeric_cols)))

# Iterate through numeric columns and create histograms
for i, col in enumerate(numeric_cols):
    sns.histplot(pre_24_rows_gt_df[col], kde=True, ax=axes[i])
    
    # Calculate 75th percentile and 25th percentile
    p75 = pre_24_rows_gt_df[col].quantile(0.75)
    p25 = pre_24_rows_gt_df[col].quantile(0.25)
    
    # Calculate upper and lower bounds for outliers
    upper_bound = p75 + 3 * pre_24_rows_gt_df[col].std()
    lower_bound = p25 - 3 * pre_24_rows_gt_df[col].std()
    
    # Add lines for upper and lower bounds
    axes[i].axvline(upper_bound, color='red', linestyle='dashed', label='Upper Bound')
    axes[i].axvline(lower_bound, color='orange', linestyle='dashed', label='Lower Bound')
    
    axes[i].set_title(f'Histogram of {col}')
    axes[i].legend()

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Assuming pre_24_rows_gt_df is your DataFrame
column_stats = pre_24_rows_gt_df.describe(include='all')

# Print mean, std, min, and max for each column
for col in pre_24_rows_gt_df.columns:
    print(f"Column: {col}")
    print(f"Mean: {column_stats.loc['mean', col]}")
    print(f"Std: {column_stats.loc['std', col]}")
    print(f"Min: {column_stats.loc['min', col]}")
    print(f"Max: {column_stats.loc['max', col]}")
    # Calculate and print 3 std + 75% and 25% - 3 std
    upper_limit = column_stats.loc['75%', col] + 3 * column_stats.loc['std', col]
    lower_limit = column_stats.loc['25%', col] - 3 * column_stats.loc['std', col]
    print(f"75% + 3 Std: {upper_limit}")
    print(f"25% - 3 Std: {lower_limit}")
    print()

### Deal with outlier by costum boundary

In [ ]:
def filter_outliers(df, reasonable_ranges, outlier_threshold=3):
    for feature, (lower_limit, upper_limit) in reasonable_ranges.items():
        # Filter outliers for each feature based on the specified range
        feature_outliers = (
            (df[feature] < lower_limit) | (df[feature] > upper_limit)
        )
        
        # Replace outliers with NaN in the original DataFrame
        df.loc[feature_outliers, feature] = np.nan

    return df


# Example of setting reasonable ranges
reasonable_ranges = {
    'peep': (0, 20),
    'fio2': (0, 100),
    'tidal_volume_observed': (0, 1250),
    'respiratory_rate_set': (0, 50),
    'plateau_pressure': (0, 40),
    'heart_rate': (25, 216),
    'sbp': (15, 252),
    'dbp': (10, 247),
    'mbp': (1, 298),
    'resp_rate': (1, 68),
    'spo2': (2, 100),
    'O2_flow': (60, 576),
    'before_weaning_hr': (0, 23),
}

# Example usage
pre_24_rows_gt_filter_df= filter_outliers(pre_24_rows_gt_df, reasonable_ranges)
pre_24_rows_gt_fill_df = fill_Nan(pre_24_rows_gt_filter_df)
# Display filtered DataFrame and outliers DataFrame
print("Filtered DataFrame:")
pre_24_rows_gt_fill_df


### Recalculate 'RSBI' and 'minute_ventilation' after dealing with outliers

In [ ]:
pre_24_rows_gt_fill_df['RSBI'] = pre_24_rows_gt_fill_df['resp_rate'] / (pre_24_rows_gt_fill_df['tidal_volume_observed'] * 0.001)
pre_24_rows_gt_fill_df['minute_ventilation'] = pre_24_rows_gt_fill_df['resp_rate'] * (pre_24_rows_gt_fill_df['tidal_volume_observed'] * 0.001)
# Apply the categorization function to create the new column
pre_24_rows_gt_fill_df['ventilator_mode_group'] = pre_24_rows_gt_fill_df['ventilator_mode'].apply(categorize_ventilator_mode)

fill with outliers(inf) as by ffill and bfill

In [ ]:
pre_24_rows_gt_fill_df['RSBI'] = pre_24_rows_gt_fill_df['RSBI'].replace(np.inf, np.nan)
pre_24_rows_gt_fill_df = fill_Nan(pre_24_rows_gt_fill_df)

In [ ]:
pre_24_rows_gt_fill_df[pre_24_rows_gt_fill_df['stay_id'] == 30109194]

### Show tableone for the Baseline features after dealing with outliers

In [ ]:
features_col = ['peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 
                'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 
                'resp_rate', 'spo2', 'O2_flow', 'RSBI', 'minute_ventilation', 'label']
categorical = []
groupby = ['label']
table_without_outlier = TableOne(pre_24_rows_gt_fill_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_without_outlier

quick look at missing val

In [ ]:
feature_for_miss = "respiratory_rate_set" # you can change this for quick look at the missing (0 records for the raw data)
# Count occurrences of 0 in feature_for_miss for each "stay_id"
counts = ventilator_setting_df[["stay_id", feature_for_miss]].groupby("stay_id").count()

# Count how many stay_id have count of 0 for feature_for_miss
zero_count_stay_id = (counts[feature_for_miss] == 0).sum()

# Display the result
print("Number of stay_id with count of 0 for respiratory_rate_set:", zero_count_stay_id)

In [ ]:
pre_24_rows_gt_fill_df

### Merge features to csv

In [ ]:
pre_24_rows_merged = pd.merge(pre_24_rows_gt_fill_df, baseline_fill_w_h_df, on=["stay_id", "label"], how="inner")
pre_24_rows_merged.drop('O2_flow', axis=1, inplace=True)

drop those having missing for all 24 rows

In [ ]:
pre_24_rows_merged.columns

In [ ]:
# Identify stay_ids with missing values
stay_ids_with_missing_values = pre_24_rows_merged.loc[pre_24_rows_merged.isnull().any(axis=1), 'stay_id'].unique()

# Create a DataFrame to store dropped rows
drop_df = pre_24_rows_merged[pre_24_rows_merged['stay_id'].isin(stay_ids_with_missing_values)].copy()

# Drop all records with specified stay_ids
pre_24_rows_merged_without_miss = pre_24_rows_merged[~pre_24_rows_merged['stay_id'].isin(stay_ids_with_missing_values)]

# Display or further process drop_df and pre_24_rows_merged_without_miss


In [ ]:
drop_df

In [ ]:
features_col = ['RSBI', 'minute_ventilation', 'ventilator_mode_group', 'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_without_miss_and_outlier = TableOne(pre_24_rows_merged_without_miss, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_without_miss_and_outlier

In [ ]:
column_order = ['subject_id', 'stay_id', 'hadm_id', 'charttime', 'before_weaning_hr', 'age_now', 'gender', 'insurance', 'race', 'first_careunit', 'admission_type', 'weight_kg', 'height_cm', 'tobacco', 'RSBI', 'minute_ventilation', 'ventilator_mode', 'ventilator_mode_group', 'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label']
len(column_order)

In [ ]:
# Reorder the DataFrame based on the desired column order
pre_24_rows_merged_ordered = pre_24_rows_merged_without_miss[column_order]
pre_24_rows_merged_ordered.to_csv("data/data_by_table/pre_24_merged_30_rows_12_07.csv", index=False)

take a look at the last hour, whether the std is smaller

In [ ]:
last_hr_df = pre_24_rows_merged_ordered[pre_24_rows_merged_ordered["before_weaning_hr"] == 0]
first_hr_df = pre_24_rows_merged_ordered[pre_24_rows_merged_ordered["before_weaning_hr"] == 23]

In [ ]:
features_col = ['RSBI', 'minute_ventilation', 'ventilator_mode_group', 'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_done = TableOne(pre_24_rows_merged_ordered, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done

In [ ]:
features_col = ['RSBI', 'minute_ventilation', 'ventilator_mode_group', 'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_done_last_hr = TableOne(last_hr_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done_last_hr

In [ ]:
features_col = ['peep', 'fio2', 'respiratory_rate_set', 'plateau_pressure', 'ventilator_mode_group', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_done_last_hr = TableOne(last_hr_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done_last_hr

In [ ]:
features_col = ['RSBI', 'minute_ventilation', 'ventilator_mode_group', 'peep', 'fio2', 'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_done_first_hr = TableOne(first_hr_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done_first_hr

In [ ]:
features_col = ['peep', 'fio2', 'respiratory_rate_set', 'plateau_pressure', 'ventilator_mode_group', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'label']
categorical = ['ventilator_mode_group']
groupby = ['label']
table_done_first_hr = TableOne(first_hr_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done_first_hr

### Jerry data for tableone

In [ ]:
pre_24h_data_v4_df = pd.read_csv("data/data_by_table/pre_24h_data_v4.csv")

In [ ]:
pre_24h_data_v4_df.columns

In [ ]:
features_col = ['peep', 'fio2', 'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'tidal_volume_observed', 'label']
categorical = []
groupby = ['label']
table_done_first_hr = TableOne(pre_24h_data_v4_df, columns=features_col, categorical=categorical, groupby=groupby, pval=True)
table_done_first_hr